# Topics in Quantitative Finance, Summer 2020 

## Lecture 7: Optimal order execution

### Tai-Ho Wang (王 太 和)   
    
<h2><img src="BCstacked_blk.jpg" align = "center" height=50 width=190></h2>

$$
\newcommand{\bea}{\begin{eqnarray}}
\newcommand{\eea}{\end{eqnarray}}
\newcommand{\supp}{\mathrm{supp}}
\newcommand{\F}{\mathcal{F} }
\newcommand{\cF}{\mathcal{F} }
\newcommand{\cG}{\mathcal{G} }
\newcommand{\E}{\mathbb{E} }
\newcommand{\Eof}[1]{\mathbb{E}\left[ #1 \right]}
\def\Cov{{ \mbox{Cov} }}
\def\Var{{ \mbox{Var} }}
\newcommand{\1}{\mathbf{1} }
\newcommand{\p}{\partial}
\newcommand{\PP}{\mathbb{P} }
\newcommand{\Pof}[1]{\mathbb{P}\left[ #1 \right]}
\newcommand{\QQ}{\mathbb{Q} }
\newcommand{\R}{\mathbb{R} }
\newcommand{\DD}{\mathbb{D} }
\newcommand{\HH}{\mathbb{H} }
\newcommand{\spn}{\mathrm{span} }
\newcommand{\cov}{\mathrm{cov} }
\newcommand{\HS}{\mathcal{L}_{\mathrm{HS}} }
\newcommand{\Hess}{\mathrm{Hess} }
\newcommand{\trace}{\mathrm{trace} }
\newcommand{\LL}{\mathcal{L} }
\newcommand{\s}{\mathcal{S} }
\newcommand{\ee}{\mathcal{E} }
\newcommand{\ff}{\mathcal{F} }
\newcommand{\hh}{\mathcal{H} }
\newcommand{\bb}{\mathcal{B} }
\newcommand{\dd}{\mathcal{D} }
\newcommand{\g}{\mathcal{G} }
\newcommand{\half}{\frac{1}{2} }
\newcommand{\T}{\mathcal{T} }
\newcommand{\bit}{\begin{itemize}}
\newcommand{\eit}{\end{itemize}}
\newcommand{\beq}{\begin{equation}}
\newcommand{\eeq}{\end{equation}}
\newcommand{\tr}{\mbox{tr}}
\newcommand{\underbar}{\underline}
$$

## Agenda

- Market impact of meta orders
- Empirical market impact estimates
- No price manipulation and allowable models
- The Euler-Lagrange equation
- Review of the Hamilton-Jacobi-Bellman equation
- The Almgren-Chriss optimal liquidation strategy
- Transient impact models
- Closed-form examples of optimal strategies


### Overview of execution algorithm design

Typically, an execution algorithm has three layers:

-   The macrotrader
    -   This highest level layer decides how to slice the order: when
        the algorithm should trade, in what size and for roughly how
        long.

-   The microtrader
    -   Given a slice of the order to trade (a child order), this level
        decides whether to place market or limit orders and at what
        price level(s).

-   The smart order router
    -   Given a limit or market order, which venue should this order be
        sent to?

In this lecture, we focus on the highest level of the algorithm: How to slice the order.

### Common algorithms

-   VWAP “Volume weighted average price”
    -   Trades at constant rate in volume time
    -   Variants of VWAP are still the most popular algos.


-   POV “Percentage of volume”
    -   Another variant of VWAP    
    -   Trades at a certain participant rate of market activity


-   TWAP “Time weighted average price”
    -   Trades at constant rate in wall clock time


-   IS “Implementation shortfall”
    -   Trades faster at the beginning and more slowly at the end
    -   The idea is to balance the risk of a worse price (which is
        proportional to quantity remaining) against the benefit of
        better execution from being more patient.



### Terminology

-   By <span>*metaorder*</span>, we mean an order that is sufficiently
    large that it cannot be filled immediately without eating into the
    order book.
    -   Nowadays, this means just about any order.
    -   Such orders need to be split.


-   We refer to each component of a metaorder as a <span>*child
    order*</span>.


-   By the <span>*metaorder impact profile*</span> (or just
    <span>*impact profile*</span>), we mean the average path of the
    stock price during and after execution of a metaorder.


-   <span>*Completion*</span> refers to the timestamp of the last child
    order of a metaorder.



### Schematic of the meta buy order impact profile

<h2><img src="http://mfe.baruch.cuny.edu/wp-content/uploads/2015/04/ShapiroPlot.png" align = "center" width=500></h2>
Figure 1: The metaorder impact profile

### The impact profile

- When a buy metaorder of length $T$ is sent, its immediate effect is to move the price upwards (to $S_T$ say).
- After completion, the price reverts to some price $S_{\infty}$ (which may be the starting price $S_0$).
- Market impact then has two components: transient and permanent.
- Knowledge of the metaorder impact profile is key to the derivation of optimal execution strategies.



### The square-root formula for market impact

-   For many years, traders have used the simple sigma-root-liquidity
    model described for example by Grinold and Kahn in 1994.


-   Software incorporating this model includes:
    -   Salomon Brothers, StockFacts Pro since around 1991
    -   Barra, Market Impact Model since around 1998
    -   Bloomberg, TCA function since 2005


-   The model is always of the rough form
    
    $$\Delta P = \text{Spread cost} +\alpha\,\sigma\,\sqrt{\frac Q V}$$
    
    where $\sigma$ is daily dollar volatility, $V$ is daily volume, $Q$ is the
    number of shares to be traded and $\alpha$ is a constant pre-factor
    of order one.



### Empirical question

Traders and trading software have been using the square-root formula
to provide a pre-trade estimate of market impact for a long time. 
Is the square-root formula empirically verified?



### Impact of proprietary metaorders (from Tóth et al.)

<a name="pic3"></a>
<h2><img src="http://mfe.baruch.cuny.edu/wp-content/uploads/2015/04/TothFigure1.png" align = "center" width=600></h2>
Figure 3: Log-log plot of the volatility adjusted price impact $\Delta P/\sigma$ vs the ratio $Q/V$

### Notes on [Figure 3](#pic3)

-   In [Figure 3](#pic3) which is taken from [Tóth et al.]<sup id="cite_ref-Toth" class="reference"><a href="#cite_note-Toth"><span>[</span>12<span>]</span></a></sup>, we see the impact of
    metaorders for CFM$^1$ proprietary trades on futures markets, in the
    period June 2007 to December 2010.

    -   Impact is measured as the average execution shortfall of a
        meta-order of size $Q$.

    -   The sample studied contained nearly 500,000 trades.


-   We see that the square-root market impact formula is verified
    empirically for meta-orders with a range of sizes spanning two to
    three orders of magnitude!

-------------------
$^1$Capital Fund Management (CFM) is a large Paris-based hedge fund.

### An explanation for square-root impact of meta orders

-   [Tóth et al.]<sup id="cite_ref-Toth" class="reference"><a href="#cite_note-Toth"><span>[</span>12<span>]</span></a></sup> present a potential explanation of the square-root market
    impact formula.


-   The visible order book represents a supply and demand profile that
    is only a small fraction of the <span>*latent*</span> supply and
    demand.


-   If latent supply and demand were linear around the mid-quote, this
    could explain the square-root law.


-   That is, if $\rho(u)$ is the latent demand profile, the change
    $\Delta P$ in the price of the underlying due to an order of size
    $Q$ should satisfy 
    
    $$\int_0^{\Delta P}\,\rho(u)\,du \sim Q.$$ 
    
    If
    $\rho(u) \sim u$ then we have $\Delta P^2 \sim Q$ which is the
    square-root law.



### Path of the stock price during execution (from Moro et al.)
<h2><img src="http://mfe.baruch.cuny.edu/wp-content/uploads/2015/04/Figure5.png" align = "center" width=400></h2>

### Empirically observed stock price path

From the preceding figures, we see that

-   There is reversion of the stock price after completion of the order.

-   Some component of the market impact of the meta order appears to be
    permanent.

-   The path of the price prior to completion looks like a power law.

    -   From [Moro et al.]<sup id="cite_ref-Moro" class="reference"><a href="#cite_note-Moro"><span>[</span>9<span>]</span></a></sup>

        \begin{eqnarray*}
        m_t-m_0 &\approx&  (4.28 \pm 0.21) \,\left(\frac{t}{T}\right)^{0.71 \pm 0.03 }\text{ (BME) } \\
        m_t-m_0 &\approx&  (2.13 \pm 0.05) \,\left(\frac{t}{T}\right)^{0.62 \pm 0.02}\text{ (LSE) } \end{eqnarray*}

        where $T$ is the duration of the meta order.



### Average price dynamics during meta order execution

- We now turn to modeling the dynamics of the stock price during meta order execution.


- It turns out (see [Gatheral]<sup id="cite_ref-Gatheral" class="reference"><a href="#cite_note-Gatheral">[6]</a></sup>) that some choices of stock price dynamics may admit price manipulation.
    - For example, square root price impact with exponential decay admits price manipulation.


- In such cases, a round-trip trade can generate cash on average.
    - You would want to repeat such a trade over and over.
    - There would be no optimal strategy.



### Price impact and slippage

- The cost of trading can be decomposed into two components:
    - The impact of our trading on the market price (the mid-price for example).
        - We refer to this effect as *price impact*.

    - Frictions such as effective bid-ask spread that affect only our execution price.
        - We refer to this effect as *slippage*. 


- In what follows, we will neglect slippage.
    - The inequality relationships we derive will all be weakened in practice to the extent that slippage becomes important.



### The principle of No Price Manipulation

A trading strategy $\Pi=\{x_t\}$ is a *round-trip trade* if

$$\int_0^T\,{\dot x}_t\,dt = x_T - x_0 = 0$$

We define a *price manipulation* to be a round-trip trade $\Pi$ whose
expected cost $C[\Pi]$ is negative.

#### <font color="red"> The principle of no price manipulation 

Price manipulation is not possible.</font>


#### <font color=blue>Corollary</font>
<font color=blue> Pump and dump schemes cannot make money on average. </font>

### Pump and Dump Schemes

(From http://www.sec.gov/answers/pumpdump.htm)

#### Definition
“Pump and dump” schemes, also known as “hype and dump manipulation”,
involve the touting of a company’s stock (typically microcap companies)
through false and misleading statements to the marketplace. After
pumping the stock, fraudsters make huge profits by selling their cheap
stock into the market.



### Statement of the problem on optimal execution

-   Given a model for the evolution of the stock price, we would
    like to find an optimal strategy for trading stock, the strategy
    that minimizes some cost function over all permissible strategies.
    
    - We will specialize to the case of stock liquidation where the initial position $x_0=X$ and the final position $x_T=0$.


-   A *deterministic* strategy is one determined in advance of trading.

-   A *dynamic* or *adapted* strategy is one that depends on the
    state of the market during execution of the order, *i.e.* on the
    stock price.

-   It will turn out that in many models, a statically optimal strategy is also dynamically optimal.

### Variational problem 

Suppose that the strategy $x_t$ minimizes the cost functional

$$C [x,\dot x]=\int_0^T\,L(t,x_t,\dot x_t)\,dt$$ 

with boundary conditions $x_0=X$, $x_T=0$ (assume we are liquidating $X$ shares). 

Let $\varphi$ be a perturbation with $\varphi(0)=\varphi(T)=0$.
The first order criterion for optimal strategy (if there exists one) is

$$\left.\frac{d}{d\epsilon}\right|_{\epsilon=0}C[x+\epsilon\,\varphi,\dot x + \epsilon\,\dot \varphi]=0.$$

### The Euler-Lagrange equation

Computing this derivative explicitly (suppressing explicit dependence on
$t$),

\begin{eqnarray*}
\left.\frac{d}{d\epsilon}\right|_{\epsilon=0}C &=& \int_0^T\,\left\{\frac{\partial L}{\partial x}\,\varphi+\frac{\partial L}{\partial\dot x}\,\dot \varphi\right\}\,dt \\
&=& \int_0^T\,\left\{\frac{\partial L}{\partial x}\,\varphi - \int_0^T \frac{d}{dt}\frac{\partial L}{\partial\dot x}\, \varphi\right\}\,dt + \left.\frac{\partial L}{\partial\dot x}\,\varphi\right|_{t=0}^T  \quad (\mbox{by applying integration by partys}) \\
&=& \int_0^T\,\left\{\frac{\partial L}{\partial x}-\frac{\partial}{\partial t}\frac{\partial L}{\partial\dot x}\right\}\,\varphi\,dt \quad (\mbox{since } \varphi(0) = \varphi(T) = 0)
\end{eqnarray*}

Since this must hold for any perturbation $\varphi$ with
$\varphi(0)=\varphi(T)=0$, we deduce


#### <font color="blue">The Euler-Lagrange equation
$$\frac{\partial}{\partial t}\,\left(\frac{\partial L}{\partial {\dot x} }\right)-\frac{\partial L}{\partial {x} }=0$$


### Stochastic control problem

A stochastic control problem is a control problem which aims to minimize certain expected costs among all admissible controls. <br>
Specifically, consider

$$
  \min_{v\in\mathcal G[0,T]} \E\left[g(X^{(v)}_T) + \int_0^T h(s,X^{(v)}_s,v_s) ds \right]
$$

where

* $g$ is referred to as the *terminal cost* and $h$ is the *running cost*


* the state variable $X^{(v)}_t$ is driven by the controlled SDE

    $$
      dX^{(v)}_t = \mu( t,X^{(v)}_t, v_t) dt + \sigma(t,X^{(v)}_t, v_t) dW_t.
    $$
    

* $\mathcal G[0,T]$ is the collection of admissible controls in the time interval $[0,T]$ 

### Bellman’s principle of optimality

“An optimal policy has the property that whatever the initial state and
initial decision are, the remaining decisions must constitute an optimal
policy with regard to the state resulting from the first decision.”

(See Bellman, 1957, Chap. III.3.)

### Value function 

* For a given admissible control $v\in\mathcal G[t,T]$, define the expected objective functional $J^{(v)}$ as
    
    $$
      J^{(v)}(t,x) = \E\left[\left. g(X^{(v)}_T) + \int_t^T h(s,X^{(v)}_s,v_s) ds \right| X_t = x \right].
    $$


* The value function $J(t,x)$ for a stochastic control problem is defined as

    $$
      J(t,x) = \min_{v\in\mathcal G[t,T]} J^{(v)}(t,x).
    $$


* The value function $J$ at $(t,x)$ is the optimal value of the control problem conditioned on the process starting at $(t,x)$ and applying the optimal control thereafter.

### Bellman's principle again

The Bellman's principle can be rephrased in terms of value function as follow. For any $0 < \epsilon < T -t$,

$$
J(t,x) = \min_{v\in\cG[t,t+\epsilon]} \E\left[\left. \int_t^{t+\epsilon} h(s,X^{(v)}_s,v_s) ds + J(t+\epsilon,X^{(v)}_{t+\epsilon}) \right| \cF_t \right].
$$

### The Hamilton-Jacobi-Bellman (HJB) equation

The value function $J$ satisfies the terminal value problem

$$
\p_t J(t,x) + \min_{v\in\cG[t]} \{\mathcal L^{(v)} J(t,x) + h(t,x,v) \} = 0, \mbox{ for } t < T
$$

with terminal condition

$$
  J(T,x) = g(x),
$$

where $\mathcal L^{(v)} := \frac{\sigma^2}2 \p_x^2 + \mu \p_x$ is the associated infinitesimal generator of the controlled process 

$$
dX^{(v)}_t = \mu( t,X^{(v)}_t, v_t) dt + \sigma(t,X^{(v)}_t, v_t) dW_t.
$$

#### Note

* The HJB equation is bascially an infinitesimal version of the Bellman's principle. 

* The optimal policy (control) is given implicitly in terms of the value function $J$.

### Almgren and Chriss

-   The seminal paper of [Almgren and Chriss]<sup id="cite_ref-AlmgrenChriss" class="reference"><a href="#cite_note-AlmgrenChriss"><span>[</span>2<span>]</span></a></sup> treats the execution of a meta
    order as a tradeoff between risk and execution cost.


-   According to their formulation:

    -   The faster an order is executed, the higher the execution cost

    -   The faster an order is executed, the lower the risk (which is
        increasing in position size).


-   Note that this is inconsistent with the empirical success of the
    square-root formula in describing the cost of meta orders.


### Almgren and Chriss

For simplicity, we consider liquidation of an existing position $X$.
Denote the position at time $t$ by $x_t$ with $x_0=X$ and $x_T=0$.

[Almgren and Chriss]<sup id="cite_ref-AlmgrenChriss" class="reference"><a href="#cite_note-AlmgrenChriss"><span>[</span>2<span>]</span></a></sup> model market impact and slippage as follows. The stock
price $S_t$ evolves as 

$$dS_t=\gamma\,dx_t+\sigma\,dZ_t,$$

where $Z_t$ is a standard Brownian motion. 

The price $\tilde S_t$ at which transactions occur is given by

$$\tilde S_t = S_t - \eta\,v_t$$

where $v_t:=-{\dot x}_t$ is the rate of
trading.


### Price path of a VWAP in the Almgren and Chriss model

<h2><img src="http://mfe.baruch.cuny.edu/wp-content/uploads/2015/04/AlmgrenChriss.png" align = "center" width=500></h2>
Figure 1: The Almgren and Chriss price path is plotted in orange.

### Consistency with empirical observation

-   This price path is inconsistent with empirical observation:
    -   The average price path during execution is linear.
    -   There is no price reversion after completion of the order.

### P&L and cost of trading of a trading strategy

Let $x_t$ be a trading strategy. The corresponding P&L (up to time $t$), denoted by $\Pi_t(x)$, is identified as 

$$
\Pi_t(x) := x_t (S_t - S_0) + \int_0^t (S_0 - \tilde S_\tau) d x_\tau.
$$

- The first term represents the (fair) value of stock shares that are yet to be transacted.

- The second term corresponds to the monetary value collected from the shares that have been transacted up to time t.

- Obviously, should there be no trade in the time interval $[0,t]$, i.e., $x_s = X$ for all $s \in [0,t]$, the P&L reads $\Pi_t(x) = X (S_t - S_0)$; reflecting the P&L from the price movement of the stock.

#### <font color=blue>Implementation shortfall as cost of trading </font>

Negative P&L is also referred to as the *implementation shortfall*, which will be used as the cost of trading, denoted by $C$ hereafter.

### P&L in Almgren-Chriss model

Note that, at the end of execution period $T$, the P&L reads
$$
  \Pi_T(x) = x_T (S_T - S_0) + \int_0^T (S_0 - \tilde S_u) d x_u,
$$
should there be $x_T$ shares yet to be transacted. Hence, in Almgren-Chriss model 

\begin{eqnarray*}
\Pi_T(x) &=& x_T (S_T - S_0) + \int_0^T (S_0 - \tilde S_u) d x_u \\
&=& \int_0^T [- \gamma (x_u - X) - \sigma Z_u + \eta v_u] d x_u \quad (\mbox{note that } x_T = 0) \\
&=& -\frac\gamma2 X^2 + \sigma \int_0^T x_u d Z_u - \eta \int_0^T v_u^2 du \quad (\mbox{Integration by parts}).
\end{eqnarray*}
Therefore, the expected cost corresponding to the trading strategy $x$ is given by  
\begin{eqnarray*}
&& \E\left[C_T(x)\right] = \E\left[-\Pi_T(x)\right] = \frac\gamma2 X^2 + \eta \int_0^T \Eof{v_u^2} du.
\end{eqnarray*}

### Expected cost of VWAP in the Almgren and Chriss model

For a VWAP, $v_t = \frac XT$ where $X$ is the total trade size and $T$ is the
duration of the order.

\begin{eqnarray*}
\E[C_T(x)] &=& \E\left[\frac\gamma2 X^2 - \sigma \int_0^T x_u d Z_u + \eta \int_0^T \left(\frac XT\right)^2 du \right] \\
&=& \frac\gamma2 X^2 + \eta \frac{X^2}T \\
&=& \left(\frac\gamma2 + \frac\eta{T}\right)X^2.
\end{eqnarray*}

The cost per share $\hat C$ of executing an order using VWAP is therefore

$$\hat{C}=\left(\frac\gamma2 + \frac\eta{T}\right)X$$

which is linear in the trade
size $X$ and inversely proportional to the duration of the order.


### The optimal strategy of a risk neutral trader

For a risk neutral trader whose objective is to minimize the cost of trading, the optimal control problem reads 

\begin{eqnarray*}
&& \min_{v} \E\left[C_T(x)\right] \\
&=& \min_{v} \left\{\frac\gamma2 X^2 + \eta \int_0^T \Eof{v_u^2} du \right\} \\
&=& \frac\gamma2 X^2 + \eta \min_v \int_0^T \Eof{v_u^2} du,
\end{eqnarray*}

where the state variable $x_t$ is driven by $dx_t = -v_t dt$ with the constraints $x_0 = X$ and $x_T = 0$.   
Note that, since $S_t$ is not involved in the last expression, we end up with a deterministic control problem which can be solved by the solution to its assocated HJB equation or, due to the simplicity of the dynamics of $x_t$, further reduced to a variational problem. 

### Variational problem 

$$\min_v \int_0^T\,v_t^2\,dt$$

with $v_t=-\dot x_t$ and $x_0 = X$, $x_T = 0$. 

The Euler-Lagrange equation is then

$$\partial_t v_t=-\partial_t^2 x_t=0$$

with boundary conditions
$x_0=X$ and $x_T=0$ and the solution is obviously

$$v_t=\frac{X}{T};\, x_t=X\,\left(1-\frac{t}{T}\right)$$

### Adding a risk term

[Almgren and Chriss]<sup id="cite_ref-AlmgrenChriss" class="reference"><a href="#cite_note-AlmgrenChriss"><span>[</span>2<span>]</span></a></sup> add a risk term that penalizes the approximate variance
of the trading cost. They claim 

$$ \mbox{Var} \left[\int_0^T\,x_t\,dS_t\right] \approx \sigma^2\,\int_0^T\,x_t^2\,dt. $$

In fact, by It&ocirc;'s isomety, it is an equality if $x_t$ is deterministic. 

The expected risk-adjusted cost is then given by (temporarily ignore the path independent term $\frac\gamma2 X^2$)

$$\eta\,\int_0^T\,{\dot x}_t^2\,dt - \lambda\,\sigma^2\,\int_0^T\,x_t^2\,dt$$

for some price of risk $\lambda$.

-   <span>Note the analogies to physics and portfolio theory.</span>

    -   <span>The first term looks like kinetic energy and the second
        term like potential energy.</span>

    -   <span>The expression looks like the objective in mean-variance
        portfolio optimization.</span>

The Euler-Lagrange equation becomes 

$$\ddot x- \kappa^2\,x=0$$ 

with

$$\kappa^2=\frac{\lambda\,\sigma^2}{\eta}.$$ 

The solution is a linear
combination of terms of the form $e^{\pm \kappa t}$ that satisfies the
boundary conditions $x_0=X$, $x_T=0$. The solution is then

<font color="red">
<span>Almgren-Chriss optimal strategy</span>

<a name="eq:optimalAC"></a>(2)
$$x(t)=X\,\frac{\sinh \kappa (T-t)}{\sinh \kappa T}$$



<font color="black">
Once again, this statically optimal solution is dynamically optimal.

- Almgren-Chriss strategy gives an example of Implementation Shortfall type: trade fast at the beginning and more slowly towards the end. 

### What happens if we change the risk term?

Suppose we penalize average VaR instead of variance. This choice of risk
term has the particular benefit of being linear in the position size.
The expected risk-adjusted cost is then given by

$$C = \eta\,\int_0^T\,{\dot x}_t^2\,dt + \lambda\,\sigma\,\int_0^T\,x_t\,dt$$

for some price of risk $\lambda$.

The Euler-Lagrange equation becomes 

$$\ddot x- A=0$$

with

$$A=\frac{\lambda\,\sigma}{2\,\eta}$$

The solution is a quadratic of the form $A\,t^2/2+B \,t + C$ that
satisfies the boundary conditions $x_0=X$, $x_T=0$. The solution is then

<a name="eq:VaRgeneral"></a>(5)
$$x(t)=\left(X-\frac{A \, T}{2}\,t\right)\,\left(1-\frac{t}{T}\right)$$

In contrast to the previous case where the cost function is monotonic
decreasing in the trading rate and the optimal choice of liquidation
time is $\infty$, in this case, we can compute an optimal liquidation
time.

When $T$ is optimal, we have
$$\frac{\partial C}{\partial T}\propto {\dot x}_T^2 + A\,x_T=0$$ from
which we deduce that $\dot x_T=0$.

Substituting into [(5)](#eq:VaRgeneral) and solving for the optimal time
$T^\star$ gives 

$$T^\star=\sqrt{\frac{2\,X}{A}}.$$

With this optimal choice $T=T^\star$, the optimal strategy becomes

\begin{eqnarray*}
x(t)&=&X\,\left(1-\frac{t}{T}\right)^2\\
u(t)&=&-\dot x(t)=2\,X\,\left(1-\frac{t}{T}\right)\end{eqnarray*}

Again, the static strategy is dynamically optimal, independent of the
stock price.

### The Almgren 2005 model

In the (2005) model of [Almgren]<sup id="cite_ref-Almgren" class="reference"><a href="#cite_note-Almgren"><span>[</span>1<span>]</span></a></sup>, the stock price $S_t$ evolves as

$$dS_t=\gamma\,d x_t+\sigma\,dZ_t$$

and the price $\tilde S_t$ at which
we transact is given by 

$$\tilde S_t = S_t - \eta\,v_t^\delta$$

where
$v_t:=-{\dot x}_t$ is the rate of trading.

The expected cost of trading is then given by

\begin{eqnarray*}
\mathcal{C} &=&\mathbb{E} \left[\int_0^T\,\tilde S_t\,v_t\,dt\right]\\
&=&\int_0^T\,(\gamma\,x_t+\eta\,v_t^\delta)\,v_t\,dt\\
&=&\frac \gamma 2\,(x_T^2-x_0^2)+\eta\,\int_0^T\,v_t^{1+\delta}\,dt\end{eqnarray*}

where wlog, we have set $S_0=0$.

We see that the first term corresponding to permanent impact is
independent of the trading strategy, as it should be. The second term is
convex in the trading rate so the minimum cost strategy is again VWAP.

### Applications of the Almgren-Chriss framework

-   Although the Almgren and Chriss price process is not particularly
    realistic, it leads to a tractable framework for solving a number of
    interesting practical problems.

-   Applications include:

    -   Portfolio liquidation

    -   Optimal liquidation with a dark pool (<span>*e.g.*</span>
        [Kratz and Schöneborn]<sup id="cite_ref-KratzSchoeneborn" class="reference"><a href="#cite_note-KratzSchoeneborn"><span>[</span>7<span>]</span></a></sup>)

    -   Optimal delta-hedging of options under transactions costs
        (<span>*e.g.*</span> [Li and Almgren]<sup id="cite_ref-LiAlmgren" class="reference"><a href="#cite_note-LiAlmgren"><span>[</span>8<span>]</span></a></sup>)

    -   Optimal liquidation in the presence of a trading signal
        (<span>*e.g.*</span> [Almgren(2012)]<sup id="cite_ref-AlmgrenSignal" class="reference"><a href="#cite_note-AlmgrenSignal"><span>[</span>3<span>]</span></a></sup>)
        

### Summary on the Almgren-Chriss model

-   The Almgren-Chriss price process is in practice the most
    widely-used.

-   It forms the basis for many of the algorithms and most of the
    thinking in algorithmic execution.

    -   despite the fact that it is unrealistic: market impact decays
        instantaneously and it is completely incompatible with the
        square-root law.

-   Because of the analytical tractability of the Almgren-Chriss
    framework, there are closed-form or quasi-closed-form solutions for
    many problems of practical interest.

### Transient impact model

The price process assumed in transient impact model is

$$S_t = S_0 + \int_0^t\,h(v_s)\,G(t-s)\,ds+M_t, \quad \mbox{ where } M_t \mbox{ is a zero mean martingale/noise.}$$


- $h(v_s)$ is referred to as the *instantaneous market impact function*, which represents the impact of trading at time $s$, and $G(t-s)$ is a decay factor. Note that $h(v) > 0$ if $v > 0$; whereas $h(v) < 0$ if $v < 0$. 


- The cumulative impact of (others’) trading is implicitly in $S_0$ and the noise term.


- The model is a generalization of processes due to Almgren, Bouchaud, and Obizhaeva and Wang.

### P&L and cost of trading in transient impact model

Note that, at the end of execution period $T$, the P&L reads
$$
  \Pi_T(x) = x_T (S_T - S_0) + \int_0^T (S_0 - \tilde S_u) d x_u,
$$
should there be $x_T$ shares yet to be transacted. Hence, in transient impact model (note that $\tilde S_t = S_t$)

\begin{eqnarray*}
\Pi_T(x) &=& x_T (S_T - S_0) + \int_0^T (S_0 - \tilde S_u) d x_u \\
&=& - \int_0^T \int_0^t h(v_u)G(t-u) du d x_t - \int_0^T M_t d x_t \quad (\mbox{since } x_T = 0) \\
&=& - \int_0^T \int_0^t h(v_u)G(t-u) du d x_t - (M_T x_T - M_0 x_0) + \int_0^T x_t d M_t \quad (\mbox{Integration by parts}) \\
&=& - \int_0^T \int_0^t h(v_u)G(t-u) du d x_t + \int_0^T x_t d M_t \quad (\mbox{since } x_T = 0 \mbox{ and } M_0 = 0).
\end{eqnarray*}
Therefore, the expected cost corresponding to the trading strategy $x$ is given by  
\begin{eqnarray*}
&& \E\left[C_T(x)\right] = \E\left[-\Pi_T(x)\right] = \Eof{\int_0^T \int_0^t h(v_u)G(t-u) du d x_t}.
\end{eqnarray*}

### The optimal strategy of a risk neutral trader

For a risk neutral trader whose objective is to minimize the cost of trading, the optimal control problem reads 

\begin{eqnarray*}
&& \min_{v} \E\left[C_T(x)\right] = \min_v \int_0^T \int_0^t h(v_u)G(t-u) du d x_t,
\end{eqnarray*}

where the state variable $x_t$ is driven by $dx_t = -v_t dt$ with the constraints $x_0 = X$ and $x_T = 0$. 
It is equivalent to a variational problem.

\begin{eqnarray*}
&& \min_{v} \E\left[C_T(x)\right] = \min_v \left\{-\int_0^T \int_0^t h(v_u)G(t-u) du v_t dt\right\},
\end{eqnarray*}

subject to the constraint $\int_0^T v_t dt = X$.

### Lagrange multiplier  

To derive the Euler-Lagrange equation, consider the Lagrangian

\begin{eqnarray*}
  && L(v,\lambda) = \int_0^T\int_0^t v_t G(t-s) h(v_s) ds dt - \lambda \left( \int_0^T v_t dt - X \right) \\
  &=& \int_0^T \left[ \int_0^t G(t-s) h(v_s) ds - \lambda \right] v_t dt + \lambda X,
\end{eqnarray*}

where $\lambda$ is the Lagrange multiplier. 

### Euler-Lagrange equation

Let $\varphi$ be a perturbation. Consider the first order criterion for the Lagrangian $L$:

\begin{eqnarray*}
  0 &=& \left.\frac{d}{d\epsilon}\right|_{\epsilon=0} L( v_t + \epsilon \varphi_t,\lambda) \\
  &=& \left.\frac{d}{d\epsilon}\right|_{\epsilon=0} \int_0^T \left[ \int_0^t G(t-s) h(v_s + \epsilon \varphi_s) ds - \lambda \right] (v_t + \epsilon \varphi_t ) dt  \\
  &=& \int_0^T \left[ \int_0^t G(t-s) h'(v_s) \varphi_s ds \right] v_t dt
   + \int_0^T \left[ \int_0^t G(t-s) h(v_s) ds - \lambda \right] \varphi_t dt  \\
  &=& \int_0^T \int_0^t G(t-s) h'(v_s) \varphi_s v_t ds dt + \int_0^T \left[ \int_0^t G(t-s) h(v_s) ds - \lambda \right] \varphi_t dt  \\
%  &=& \int_0^T \int_s^T G(t-s) h'(v_s) \varphi_s v_t dt ds + \int_0^T \left[ \int_0^t G(t-s) h(v_s) ds + \lambda \right] \varphi_t dt  \\
  &=& \int_0^T \left[ \int_t^T G(s-t) h'(v_t) v_s ds + \int_0^t G(t-s) h(v_s) ds - \lambda \right] \varphi_t dt.
\end{eqnarray*}

Since $\varphi_t$ is arbitrary (because the first order criterion must hold for any perturbation), we must have

<a name="eq:intEL-TH"></a>(2)
<font color=blue>
$$
  \int_t^T G(s-t) h'(v_t) v_s ds + \int_0^t G(t-s) h(v_s) ds = \lambda 
$$
</font>
<br>
for all $0 \leq t \leq T$. This a generalized Fredholm integral equation of the first kind. The Lagrange multiplier $\lambda$ is determined by the constraint $\int_0^T v_t dt = X$.

### Obizhaeva and Wang 2005

The model in [Obizhaeva and Wang]<sup id="cite_ref-ObizhaevaWang" class="reference"><a href="#cite_note-ObizhaevaWang"><span>[</span>7<span>]</span></a></sup> is given by

<a name="eq:OWprocess"></a>(1)
$$S_t = S_0 - \eta\,\int_0^t\,v_s\,e^{-\rho\,(t-s)}\,ds + \int_0^t\,\sigma\,dZ_s$$

with $v_t=-\dot x_t$.

- Market impact decays exponentially, *i.e.*, $G(\tau) = e^{-\rho\tau}$ 

- Instantaneous market impact is linear in the rate of trading, $h(v) = -\eta v$.


The expected cost of trading becomes:
$$\mathcal{C} =\eta\,\int_0^T\int_0^t\,{v}_s\,e^{-\rho\,(t-s)}\,ds\, v_tdt$$

### The Obizhaeva and Wang average price path
<h2><img src="http://mfe.baruch.cuny.edu/wp-content/uploads/2015/04/ObizhaevaWang.png" align = "center" width=500></h2>
Figure 1: The Obizhaeva Wang average price path: higher $\rho$ in pink; lower $\rho$ in brown.

The Euler-Lagrange equation [(2)](#eq:intEL-TH) in this case reads:

<a name="eq:intEL"></a>(2')
$$
- \eta \int_t^T\,v_s\,e^{-\rho\,(s-t)}\,ds - \eta \int_0^t\,v_s\,e^{-\rho\,(t-s)}\,ds  = \lambda.
$$

[(2')](#eq:intEL) may be rewritten as

$$\int_0^T\,v_s\,e^{-\rho\,|t-s|}\,ds=-\frac\lambda\eta$$

which is a Fredholm integral equation of the first kind.

### Optimality and price manipulation

-   <span>For all of the models considered so far, there was an optimal
    strategy.</span>


-   <span>The optimal strategy always involved trades of the same sign.
    So no sells in a buy program, no buys in a sell program.</span>


-   <span>Recall that [Gatheral]<sup id="cite_ref-Gatheral" class="reference"><a href="#cite_note-Gatheral"><span>[</span>4<span>]</span></a></sup> showed that we can write down models for
    which price manipulation is possible.</span>

    -   <span>For example, square root price impact with exponential
        decay admits price manipulation.</span>


-   <span>In such cases, a round-trip trade can generate cash on
    average.</span>

    -   <span>You would want to repeat such a trade over and
        over.</span>

    -   <span>There would be no optimal strategy.</span>

### Linear transient market impact

In this case, the impact function is linear, i.e., $h(v) = \eta v$. When the admissible
strategy $x$ is used, the price $S_t$ is given by

<a name="eq:ModelEquation"></a>(5)
$$S_t = S_0 + \eta \int_0^t {G}(t-s)\,dx_s + \mbox{ martingale},$$

and the expected cost of liquidation is given by

<a name="eq:quadraticform"></a>(6)
$$\mathcal{C} (x):=\frac{\eta}{2}\,\int_0^T\int_0^T {G}(|t-s|)\,dx_s\,dx_t.$$


### Condition for no price manipulation

#### <font color="blue">Definition (Huberman and Stanzl)
<font color="blue">

A *round
trip* is an admissible strategy with $x_0=x_T$. A *price manipulation
strategy* is a round trip with strictly negative expected costs.</span>

#### <font color="red">Theorem (Bochner)
<font color="red"><span>$\mathcal{C} (x)\ge0$ for all admissible
strategies $x$ if and only if ${G}(|\cdot|)$ can be represented as the
Fourier transform of a positive finite Borel measure $\mu$ on
$\mathbb{R} $, i.e., $${G}(|\tau|)=\int e^{i \,z\,\tau}\,\mu(dz).$$</span>

### First order condition

For linear market impact, the Euler-Lagrange equation [(2)](#eq:intEL-TH) reduces to 

$$\int_0^T {G}(|t-s|)\,dx_s =\lambda\qquad\text{for all $t\in[0,T] $.}$$

Moreover, we have

#### <font color="red">Theorem
<font color="red">
Suppose that ${G}$ is positive definite. Then $x^*$
minimizes $\mathcal{C} (\cdot)$ if and only if there is a constant
$\lambda$ such that $x^*$ solves the generalized Fredholm integral
equation

<a name="eq:Fredholm"></a>(7)
$$\int_0^T {G}(|t-s|)\,dx^*_s =\lambda\qquad\text{for all $t\in[0,T] $.}$$

In this case, $\mathcal{C} (x^*)=\frac{1}{2}\,\lambda \,X$. In
particular, $\lambda$ must be nonzero as soon as ${G}$ is strictly
positive definite and $X\neq0$.

### Example I: Linear market impact with exponential decay

$G(\tau)=e^{-\rho\,\tau}$ and the optimal strategy $u(s)$ solves

$$\int_0^T\,u(s)e^{-\rho\,|t-s|}\,ds=\text{const.}$$

The solution is 

$$u(t)=A\,\left\{\delta(t)+\rho+\delta(T-t)\right\}$$

The normalizing
factor $A$ is given by

$$\int_0^T\,u(t)\,dt=X=A\, \left(2+\rho\,T\right)$$

The optimal strategy
consists of block trades at $t=0$ and $t=T$ and continuous trading at
the constant rate $\rho$ between these two times.

### Schematic of optimal strategy

The optimal strategy with $\rho=0.1$ and $T=1$
<h2><img src="http://mfe.baruch.cuny.edu/wp-content/uploads/2015/04/Algo2010Exponential.png" align = "center" width=500></h2>

### Example II: Linear market impact with power-law decay

$G(\tau)=\tau^{-\gamma}$ and the optimal strategy $u(s)$ solves

$$\int_0^T\,\frac{u(s)}{|t-s|^\gamma}\,ds=\text{const.}$$

The solution
is 

$$u(s)=\frac{A}{\left[s\,(T-s)\right]^{(1-\gamma)/2}}$$

The
normalizing factor $A$ is given by

$$\int_0^T\,u(t)\,dt=X=A\, \sqrt{\pi }\,\left(\frac{T}{2}\right)^\gamma \,\frac{ \Gamma \left(
\frac{1+\gamma}{2}\right)}{\Gamma \left(1+\frac{\gamma }{2}\right)}$$

The optimal strategy is absolutely continuous with no block trades.
However, it is singular at $t=0$ and $t=T$.

### Schematic of optimal strategy

The red line is a plot of the optimal strategy with $T=1$ and
$\gamma=1/2$.
<h2><img src="http://mfe.baruch.cuny.edu/wp-content/uploads/2015/04/Algo2010PowerLaw.png" align = "center" width=500></h2>

### When is the optimal strategy bucket-shaped?

-   For which nonincreasing convex decay kernels $G$ does the optimal
    execution strategy have singular components only at $t=0$ and $t=T$
    with absolutely continuous trading on the interval $(0,T)$?

    -   This question is answered in Corollary 1 of [Alfonsi and Schied]<sup id="cite_ref-AlfonsiSchied" class="reference"><a href="#cite_note-AlfonsiSchied"><span>[</span>1<span>]</span></a></sup>.

<font color="red">
#### Corollary
Suppose that $G$ is completely monotone with
$G''(0+):=\lim_{t \downarrow 0} G''(t) < \infty$. Then the capacitary
measure $\mu^\star$ has two Dirac components at $t=0$ and $t=T$ and has
a continuous Lebesgue density on $(0,T)$.

### Completely monotone functions

#### <font color="blue"> Definition
$G$ is completely monotone if $G$ belongs to $C^\infty((0,\infty))$ and
$(-1)^nG^{(n)}$ is nonnegative in $(0,\infty)$ for $n \in \mathbb N$.
</font>

#### <font color=red> Theorem (Hausdorff–Bernstein–Widder)
G is completely monotone if and only it is
the Laplace transform of a Borel probability measure $\lambda$ on
$\mathbb{R}+$:

$$G(t) = \int \,e^{-\rho\,t}\,\lambda(d\rho), \quad t \geq 0.$$


### Summary 

-   <span>The optimal trading strategy depends on the model.</span>

    -   <span>For Almgren-Chriss style models, if the price of risk is
        zero, the minimal cost strategy is VWAP.</span>

    -   <span>In Obizhaeva-Wang style models, the minimal cost strategy is
        to trade a block at inception, a block at completion and at a
        constant rate in between.</span>

### References

<br />

<div class="reflist" style="list-style-type: decimal;">

<ol>

  <li id="cite_note-Almgren"><span class="mw-cite-backlink"><b><a href="#cite_ref-Almgren">^</a></b></span>Robert Almgren, Chee Thum, Emmanuel Hauptmann, and Hong Li, Equity
market impact, <span>*Risk*</span> July 2005, 57–62.
  </li>

  <li id="cite_note-Bershova"><span class="mw-cite-backlink"><b><a href="#cite_ref-Bershova">^</a></b></span>Nataliya Bershova and Dmitry Rakhlin, The Non-Linear Market Impact of
Large Trades: Evidence from Buy-Side Order Flow, <span>*Quantitative
Finance*</span> <span>**13**</span>(11) 1759–1778 (2013).
  </li>
  
  <li id="cite_note-BMP"><span class="mw-cite-backlink"><b><a href="#cite_ref-BMP">^</a></b></span>Jean-Philippe Bouchaud, Marc Mézard and Marc Potters, Statistical
properties of stock order books: empirical results and models,
<span>*Quantitative Finance*</span> <span>**2**</span>(4) 251–256
(2002).
  </li>
  
  <li id="cite_note-Donier"><span class="mw-cite-backlink"><b><a href="#cite_ref-Donier">^</a></b></span>Jonathan Donier, Market Impact with Autocorrelated Order Flow under
Perfect Competition, <span>*SSRN*</span> available at
<http://ssrn.com/abstract=2191660> (2012).
  </li>
  
  <li id="cite_note-FGLW"><span class="mw-cite-backlink"><b><a href="#cite_ref-FGLW">^</a></b></span>J Doyne Farmer, Austin Gerig, Fabrizio Lillo, and Henri Waelbroeck, How
efficiency shapes market impact, available at <span>*Quantitative
Finance*</span> <span>**13**</span>(11) 1743–1758 (2013).
  </li>
  
  <li id="cite_note-Gatheral"><span class="mw-cite-backlink"><b><a href="#cite_ref-Gatheral">^</a></b></span>Jim Gatheral, No-dynamic-arbitrage and market impact,
<span>*Quantitative Finance*</span> <span>**10**</span>(7) 749–759
(2010).
  </li>

  <li id="cite_note-GSS"><span class="mw-cite-backlink"><b><a href="#cite_ref-GSS">^</a></b></span>Jim Gatheral, Alexander Schied, and Alla Slynko, Exponential resilience
and decay of market impact, in <span>*Econophysics of Order-driven
Markets*</span>, Springer, 225–236, (2011).
  </li>
  
  <li id="cite_note-LilloMikeFarmer"><span class="mw-cite-backlink"><b><a href="#cite_ref-LilloMikeFarmer">^</a></b></span>Fabrizio Lillo, Szabolcs Mike, and J Doyne Farmer, Theory for long
memory in supply and demand, <span>*Phys. Rev. E*</span>
<span>**71**</span>(6) 66122 (2005).
  </li>
  
  <li id="cite_note-Moro"><span class="mw-cite-backlink"><b><a href="#cite_ref-Moro">^</a></b></span>Esteban Moro, Javier Vicente, Luis G Moyano, Austin Gerig, J. Doyne
Farmer, Gabriella Vaglica, Fabrizio Lillo, and Rosario N Mantegna,
Market impact and trading profile of meta orders in stock markets,
<span>*Physical Review E*</span> <span>**80**</span>(6) 066102 (2009).
  </li>
  
  <li id="cite_note-ObizhaevaWang"><span class="mw-cite-backlink"><b><a href="#cite_ref-ObizhaevaWang">^</a></b></span>Anna Obizhaeva and Jiang Wang, Optimal trading strategy and
supply/demand dynamics <span>*Journal of Financial Markets*</span>
<span>**16**</span>(1) 1–32 (2013).
  </li>
  
  <li id="cite_note-SFGK"><span class="mw-cite-backlink"><b><a href="#cite_ref-SFGK">^</a></b></span>E Smith, JD Farmer, L Gillemot, and S Krishnamurthy, Statistical theory
of the continuous double auction, <span>*Quantitative Finance*</span>
<span>**3**</span>(6) 481–514 (2003).
  </li>
  
  <li id="cite_note-Toth"><span class="mw-cite-backlink"><b><a href="#cite_ref-Toth">^</a></b></span>Bence Tóth, Yves Lempérière, Cyril Deremble, Joachim de Lataillade,
Julien Kockelkoren, and Jean-Philippe Bouchaud, Anomalous price impact
and the critical nature of liquidity in financial markets,
<span>*Physical Review X*</span> 021006, 1-11(2011).
  </li>
  
  <li id="cite_note-WaelbroeckGomes"><span class="mw-cite-backlink"><b><a href="#cite_ref-WaelbroeckGomes">^</a></b></span>Henri Waelbroeck and Carla Gomes, Is Market Impact a Measure of the
Information Value of Trades? Market Response to Liquidity vs. Informed
Trades, <span>*SSRN*</span> available at
<http://ssrn.com/abstract=2291720> (2013).
  </li>

  
</ol>